Reference:

[**Introduction to LangChain for Agentic AI**](https://courses.analyticsvidhya.com/courses/take/introduction-to-langchain-for-agentic-ai/lessons/61748853-course-introduction) 

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install -qq langchain
!pip install -qq langchain-openai
!pip install -qq langchain-community
!pip install -qq langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.53.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
langchain-huggingface 0.3.0 requires langchain-core<1.0.0,>=0.3.65, but you have langchain-core 0.3.63 which is incompatible.


## Enter Open AI API Key

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [4]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

## Multi-user Conversation Chains with ChatMessageHistory and RunnableWithMessageHistory

The concept of `ChatHistory` refers to a class in LangChain which can be used to wrap an arbitrary chain. This `ChatHistory` will keep track of inputs and outputs of the underlying chain, and append them as messages to a message database. Future interactions will then load those messages and pass them into the chain as part of the input.

The beauty of `ChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in memory. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain


- `ChatMessageHistory`: This is a class in LangChain that stores a sequence of chat messages (such as HumanMessage, AIMessage, SystemMessage) for a conversation. It allows you to append new messages and retrieve the full message history, which is useful for maintaining conversational context.

- `BaseChatMessageHistory`: This is an abstract base class that defines the interface for chat message history storage backends. It specifies methods like add_message, get_messages, and clear, which concrete implementations (like ChatMessageHistory, RedisChatMessageHistory, etc.) must provide.

- `RunnableWithMessageHistory`: This is a wrapper for a Runnable (such as a chain or LLM) that automatically manages message history for each session or user. It uses a function (like get_session_history) to retrieve the appropriate message history object based on a session ID, and injects the history into the chain's input. This enables multi-user or multi-session conversational experiences.

- `MessagesPlaceholder`: This is a special prompt template variable in LangChain that acts as a placeholder for a list of messages (e.g., the conversation history). When rendering a prompt, MessagesPlaceholder is replaced with the actual messages from history, allowing the LLM to see the full conversation context.


In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# used to retrieve conversation history from memory
# based on a specific user or session ID
history_store = {}
def get_session_history(session_id):
    if session_id not in history_store:
        history_store[session_id] = ChatMessageHistory()
    return history_store[session_id]

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

# create a basic LLM Chain
llm_chain = (prompt_template
                |
             chatgpt)

# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    for chunk in conv_chain.stream({"human_input": prompt},
                                   {'configurable': { 'session_id': session_id}}):
        print(chunk.content, end="")

In [10]:
user_id = 'Sourav'
prompt = "Hi I am Sourav, can you explain AI in 3 bullet points?"
chat_with_llm(prompt, user_id)

Of course, Sourav! Here are three key points about AI:

1. **Core Concept**: Artificial Intelligence (AI) involves creating computer systems that can perform tasks typically requiring human intelligence, such as learning, reasoning, problem-solving, and understanding language. It aims to mimic cognitive functions.

2. **Machine Learning and Deep Learning**: A significant subset of AI is machine learning, where algorithms learn from data to improve their performance over time. Deep learning, a further subset, uses neural networks to analyze complex data patterns, enabling advancements in areas like image and speech recognition.

3. **Real-World Applications**: AI is transforming various sectors, including healthcare (for predictive analytics and personalized medicine), finance (for algorithmic trading and risk assessment), and customer service (through chatbots and virtual assistants). Its growing presence raises important discussions about ethics, accountability, and the future of work

In [11]:
prompt = "Now do the same for deep learning"
chat_with_llm(prompt, user_id)

Sure, Sourav! Here are three key points about deep learning:

1. **Definition and Structure**: Deep learning is a subset of machine learning that uses artificial neural networks with multiple layers (hence "deep") to model complex patterns in large datasets. These networks are designed to automatically learn representations of data, making them particularly effective for tasks like image and speech recognition.

2. **How It Works**: Deep learning algorithms process data through layers of interconnected nodes (neurons), where each layer extracts increasingly abstract features. For example, in image recognition, initial layers might detect edges, while deeper layers identify shapes and objects. This hierarchical learning enables the model to understand intricate patterns without manual feature extraction.

3. **Applications and Impact**: Deep learning has led to significant advancements in various fields, including computer vision (self-driving cars), natural language processing (languag

In [12]:
prompt = "Discuss briefly what have we discussed so far is bullet points?"
chat_with_llm(prompt, user_id)

Certainly! Here’s a brief summary of what we’ve discussed so far in bullet points:

### Artificial Intelligence (AI)
- **Definition and Purpose**: AI simulates human intelligence in machines to perform tasks like problem-solving and language understanding.
- **Types of AI**: Divided into Narrow AI (specific tasks) and General AI (human-like intelligence), with most applications currently being Narrow AI.
- **Applications and Impact**: Used in various industries (healthcare, finance, transportation) to drive efficiency and innovation, while raising ethical considerations.

### Deep Learning
- **Definition and Structure**: A subset of machine learning that uses multi-layered neural networks to model complex patterns in large datasets.
- **How It Works**: Processes data through interconnected layers, extracting increasingly abstract features for tasks like image and speech recognition.
- **Applications and Impact**: Significant advancements in fields like computer vision and natural langu

In [13]:
user_id = 'Arnab'
prompt = "Hi can you explain what is an LLM in 2 bullet points?"
chat_with_llm(prompt, user_id)

Sure! Here are two key points about LLMs (Large Language Models):

- **Definition**: LLMs are advanced artificial intelligence models designed to understand, generate, and manipulate human language by processing vast amounts of text data, enabling them to perform tasks like translation, summarization, and conversation.

- **Architecture**: They typically use deep learning techniques, particularly transformer architectures, which allow them to capture complex patterns and relationships in language, making them highly effective for various natural language processing applications.

In [14]:
prompt = "Actually I meant in the context of AI?"
chat_with_llm(prompt, user_id)

Got it! Here’s a refined explanation of LLMs in the context of AI:

- **Large Language Models (LLMs)**: These are AI models that are trained on extensive datasets containing text from books, articles, and websites, enabling them to understand and generate human-like text based on the input they receive.

- **Applications**: LLMs are used in various applications, including chatbots, content creation, language translation, and more, leveraging their ability to comprehend context and produce coherent and contextually relevant responses.

In [15]:
prompt = "Summarize briefly what we have discussed so far?"
chat_with_llm(prompt, user_id)

We discussed Large Language Models (LLMs) in the context of AI, highlighting two main points: 

1. LLMs are advanced AI models trained on vast text datasets to understand and generate human-like language.
2. They are utilized in various applications, such as chatbots and content creation, due to their ability to comprehend context and produce relevant responses.

In [16]:
user_id = 'Sourav'
prompt = "Summarize briefly what we have discussed so far?"
chat_with_llm(prompt, user_id)

We have discussed two key topics: Artificial Intelligence (AI) and Deep Learning.

1. **Artificial Intelligence (AI)**: 
   - AI simulates human intelligence to perform tasks like problem-solving and language understanding.
   - It is categorized into Narrow AI (specific tasks) and General AI (human-like intelligence), with most current applications being Narrow AI.
   - AI is applied across various industries, enhancing efficiency and innovation, while also raising ethical concerns.

2. **Deep Learning**: 
   - Deep learning is a subset of machine learning that utilizes multi-layered neural networks to analyze complex data patterns.
   - It works by processing data through layers that extract increasingly abstract features, making it effective for tasks like image and speech recognition.
   - Deep learning has led to advancements in fields such as computer vision and natural language processing, but it also presents challenges related to interpretability and resource demands.

If you 

In [17]:
user_id = 'Arnab'
prompt = "Summarize briefly what we have discussed so far?"
chat_with_llm(prompt, user_id)

We discussed Large Language Models (LLMs) in AI, noting that they are advanced models trained on extensive text data to understand and generate human-like language. We also highlighted their applications, including chatbots and content creation, due to their contextual comprehension and response generation capabilities.